In [1]:
!pip install selenium


In [2]:
import selenium
from selenium import webdriver
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = r'C:\Users\navee\Downloads\chromedriver_win32\chromedriver.exe'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

C:\Users\navee\AppData\Local\Temp\ipykernel_27832\1192264339.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)


In [ ]:
wd.get('https://google.com')

In [4]:
wd.quit()

In [24]:
!pip install requests
import time
import io
import requests
from PIL import Image
import hashlib
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [25]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [28]:
import os
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=10):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [29]:
search_term='Acute lymphoblastic leukemia'
search_and_download(search_term=search_term,driver_path=DRIVER_PATH)


C:\Users\navee\AppData\Local\Temp\ipykernel_27832\3097677613.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
C:\Users\navee\AppData\Local\Temp\ipykernel_27832\4239652644.py:25: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


C:\Users\navee\AppData\Local\Temp\ipykernel_27832\4239652644.py:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 10 image links, done!
SUCCESS - saved https://upload.medbullets.com/topic/120467/images/all_-_peripherial_blood_-_diagnosis_-_01.jpg - as ./images\acute_lymphoblastic_leukemia\908ac7603b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRVnTsFHaI4daF_Sf0nPZVS_40Cew38kI858g&usqp=CAU - as ./images\acute_lymphoblastic_leukemia\944869298f.jpg
SUCCESS - saved https://stjude.scene7.com/is/image/stjude/histology-animation-illustration?fmt=png-alpha&wid=500 - as ./images\acute_lymphoblastic_leukemia\eee65aae74.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/thumb/0/0e/Acute_leukemia-ALL.jpg/300px-Acute_leukemia-ALL.jpg - as ./images\acute_lymphoblastic_leukemia\5a81acf4b1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT9rZgj4IdrJqhAPjlb2HsDDDkj9tm70sokhw&usqp=CAU - as ./images\acute_lymphoblastic_leukemia\bff57bc31c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSIIfrVSQ6kflykZlFhqfyWOhUiUhu